In [1]:
from sqlalchemy import create_engine
import pandas as pd

user = 'jupyter_user'
password = '666-Zelda'
host = 'localhost'
port = '3306'
database = 'football'

engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")

In [2]:
engine.connect()

In [3]:
with open('Football.sql', 'r', encoding='utf-8') as f:
    sql_script = f.read()

import mysql.connector

conn = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

cursor = conn.cursor()
for statement in sql_script.split(';'):
    if statement.strip():
        cursor.execute(statement)
conn.commit()
cursor.close()
conn.close()

In [4]:
df = pd.read_sql("SELECT * FROM players", engine)
df

,player_id,team_id,name,position,date_of_birth,nationality
0,2,264,Juanmi,Offence,1993-05-20,Spain
1,8,90,Willian Jose,Offence,1991-11-23,Brazil
2,9,100,Diego Llorente,Defence,1993-08-16,Spain
3,11,92,Alvaro Odriozola,Defence,1995-12-14,Spain
4,14,586,Antonio Sanabria,Offence,1996-03-04,Paraguay
...,...,...,...,...,...,...
3145,252916,455,Luca Boncori,Offence,2006-02-21,Italy
3146,252918,450,Junior Ajayi,Offence,2004-10-11,Ivory Coast
3147,252993,558,Yoel Lago,Midfield,2004-03-25,Spain
3148,253001,558,Javi Rueda,Defence,2002-05-08,Spain


In [5]:
player_nationality = pd.read_sql('SELECT * FROM players WHERE nationality = "Scotland"', engine)
player_nationality

,player_id,team_id,name,position,date_of_birth,nationality
0,4359,356,Oliver McBurnie,Offence,1996-06-04,Scotland
1,5552,389,Jacob Brown,Offence,1998-04-10,Scotland
2,7846,55,Fraser Hornby,Midfield,1999-09-13,Scotland
3,7868,64,Andrew Robertson,Defence,1994-03-11,Scotland
4,7905,66,Scott McTominay,Midfield,1996-12-08,Scotland
5,7930,67,Matt Ritchie,Defence,1989-09-10,Scotland
6,15615,92,Kieran Tierney,Defence,1997-06-05,Scotland
7,34495,1044,Ryan Christie,Midfield,1995-02-22,Scotland
8,34638,103,Lewis Ferguson,Midfield,1999-08-24,Scotland
9,34646,58,John McGinn,Midfield,1994-10-18,Scotland


In [6]:
player_by_team = pd.read_sql("SELECT p.name,p.position,p.nationality,p.date_of_birth,t.name AS team_name FROM players p LEFT JOIN teams t ON p.team_id = t.team_id ORDER BY t.name", engine)
player_by_team

,name,position,nationality,date_of_birth,team_name
0,Seedy Jarju,Defence,Gambia,2004-10-28,1. FC Heidenheim 1846
1,Luka Janes,Midfield,Germany,2004-01-19,1. FC Heidenheim 1846
2,Elidon Qenaj,Offence,Germany,2003-05-22,1. FC Heidenheim 1846
3,Christopher Negele,Offence,Germany,2005-04-11,1. FC Heidenheim 1846
4,Adrian Beck,Midfield,Germany,1997-06-09,1. FC Heidenheim 1846
...,...,...,...,...,...
3145,Craig Dawson,Defence,England,1990-05-06,Wolverhampton Wanderers
3146,Thomas King,Goalkeeper,Wales,1995-03-09,Wolverhampton Wanderers
3147,Pedro Neto,Offence,Portugal,2000-03-09,Wolverhampton Wanderers
3148,Hwang Heechan,Offence,South Korea,1996-01-26,Wolverhampton Wanderers


In [7]:
df['nationality'].value_counts()

nationality
Spain         506
France        402
Germany       319
England       300
Italy         299
             ... 
Andorra         1
Burundi         1
Panama          1
Honduras        1
Luxembourg      1
Name: count, Length: 107, dtype: int64

In [8]:
immigrants = pd.read_sql(
    """
    WITH player_data AS (
        SELECT 
            p.player_id,
            p.name AS player_name,
            p.nationality,
            t.team_id,
            t.name AS team_name,
            l.league_id,
            l.name AS league_name,
            CASE
                WHEN l.name = 'Premier League' AND p.nationality IN ('England', 'Wales') THEN 0
                WHEN l.name = 'Serie A' AND p.nationality = 'Italy' THEN 0
                WHEN l.name = 'La Liga' AND p.nationality = 'Spain' THEN 0
                WHEN l.name = 'Bundesliga' AND p.nationality = 'Germany' THEN 0
                WHEN l.name = 'Ligue 1' AND p.nationality = 'France' THEN 0
                ELSE 1
            END AS is_immigrant
        FROM players p
        JOIN teams t ON p.team_id = t.team_id
        JOIN leagues l ON t.league_id = l.league_id
    ),
    league_stats AS (
        SELECT 
            league_name,
            COUNT(*) AS total_players,
            SUM(is_immigrant) AS immigrant_players,
            ROUND(100 * SUM(is_immigrant) / COUNT(*), 2) AS immigrant_percentage
        FROM player_data
        GROUP BY league_name
    ),
    team_stats AS (
        SELECT 
            league_name,
            team_name,
            COUNT(*) AS team_total_players,
            SUM(is_immigrant) AS team_immigrant_players,
            ROUND(100 * SUM(is_immigrant) / COUNT(*), 2) AS team_immigrant_percentage,
            RANK() OVER (PARTITION BY league_name ORDER BY ROUND(1.0 * SUM(is_immigrant) / COUNT(*), 4) DESC) AS team_rank
        FROM player_data
        GROUP BY league_name, team_name
    )
    SELECT 
        ls.league_name,
        ls.total_players,
        ls.immigrant_players,
        ls.immigrant_percentage,
        ts.team_name AS most_affected_team,
        ts.team_total_players,
        ts.team_immigrant_players,
        ts.team_immigrant_percentage
    FROM league_stats ls
    JOIN team_stats ts ON ls.league_name = ts.league_name
    WHERE ts.team_rank = 1
    ORDER BY ls.immigrant_percentage DESC
    """
, engine)
immigrants

,league_name,total_players,immigrant_players,immigrant_percentage,most_affected_team,team_total_players,team_immigrant_players,team_immigrant_percentage
0,Serie A,673,388.0,57.65,Udinese Calcio,34,28.0,82.35
1,Premier League,636,361.0,56.76,Fulham,29,23.0,79.31
2,Ligue 1,554,273.0,49.28,Olympique Lyonnais,29,20.0,68.97
3,Bundesliga,555,262.0,47.21,Bayer 04 Leverkusen,29,20.0,68.97
4,La Liga,694,246.0,35.45,Real Madrid CF,36,18.0,50.00


In [12]:
oldest_by_nationality = pd.read_sql(
    """
    SELECT p.*, t.name FROM players p
    JOIN teams t ON t.team_id = p.team_id 
    WHERE age = (select max(p2.age) from players p2 where p2.nationality = p.nationality)
    """
, engine)
oldest_by_nationality

,player_id,team_id,name,position,date_of_birth,nationality,age,name
0,317,2,Pavel Kaderabek,Defence,1992-04-25,Czech Republic,33,TSG 1899 Hoffenheim
1,6696,3,Lukas Hradecky,Goalkeeper,1989-11-24,Finland,35,Bayer 04 Leverkusen
2,7963,5,Maxim Choupo-Moting,Offence,1989-03-23,Cameroon,36,FC Bayern Munchen
3,158,10,Mahmoud Dahoud,Midfield,1996-01-01,Syria,29,VfB Stuttgart
4,193091,10,Raul Paula,Midfield,1976-06-26,Germany,48,VfB Stuttgart
...,...,...,...,...,...,...,...,...
111,9533,721,Peter Gulacsi,Goalkeeper,1990-05-06,Hungary,35,RB Leipzig
112,9550,721,Kevin Kampl,Midfield,1990-10-09,Slovenia,34,RB Leipzig
113,4203,1044,Darren Randolph,Goalkeeper,1987-05-12,Ireland,38,AFC Bournemouth
114,121808,5890,Lameck Banda,Offence,2001-01-29,Zambia,24,US Lecce


In [13]:
oldest_by_nationality.to_csv('oldest_by_nationality.csv')